In [ ]:
import networkx as nx 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sys
import seaborn as sns
import json
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.tokenize import sent_tokenize, word_tokenize
from wordcloud import WordCloud, STOPWORDS

#This section imports all libraries to be used

data1 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=1&api_key=RGAPI-9a80f68b-f6e7-4111-bee8-3f93e703dd90"
                  )
data2 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=2&api_key=RGAPI-9a80f68b-f6e7-4111-bee8-3f93e703dd90"
                  )
data3 = pd.read_json(
    'https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=3&api_key=RGAPI-9a80f68b-f6e7-4111-bee8-3f93e703dd90'
)

data4 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page=1&api_key=RGAPI-9a80f68b-f6e7-4111-bee8-3f93e703dd90"
                  )
data5 = pd.read_json(
                     "https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page=2&api_key=RGAPI-9a80f68b-f6e7-4111-bee8-3f93e703dd90"
                  )
data6 = pd.read_json(
    'https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/IV?page=3&api_key=RGAPI-9a80f68b-f6e7-4111-bee8-3f93e703dd90'
)

#This accesses the API for all data

d1 = pd.DataFrame(data1)
d2 = pd.DataFrame(data2)
d3 = pd.DataFrame(data3)

d4 = pd.DataFrame(data4)
d5 = pd.DataFrame(data5)
d6 = pd.DataFrame(data6)

silver_frames = [d1, d2, d3]
gold_frames = [d4, d5, d6]

silver = pd.concat(silver_frames, ignore_index=True)
gold = pd.concat(gold_frames, ignore_index=True)

#this puts the data into the two dataframes, silver and gold, based on the different ranks, and then combines them into a total dataset in the next section

frames = [silver, gold]
total = pd.concat(frames, ignore_index=True)
total

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,miniSeries
0,de901a1b-c840-455c-b553-2a332f8ff0b1,RANKED_SOLO_5x5,SILVER,I,AmEWdiCcCsgjFOc5WMnyNskLYaQmEX0x5rM2GCcTJc7RKSFH,ComsicGhostRider,96,155,147,False,False,True,True,NaN
1,add324d7-b028-4af7-b156-dc1b48489677,RANKED_SOLO_5x5,SILVER,I,L-LuEfE0GlILDM68KVvXXZScsI-ON67byoKOAIdGNB5MDe27,Lowfyz,44,36,35,False,False,False,False,NaN
2,df331365-7ffa-4fcb-81d5-b1959a848f66,RANKED_SOLO_5x5,SILVER,I,-Ga4FnlH2r5ec0JmdxnC_EBY_M1mtuhvZDv8QPFxbomChOYp,Subt3rfug3,23,38,35,False,False,False,False,NaN
3,35192393-9d6e-4b88-a3c2-6fd945899f11,RANKED_SOLO_5x5,SILVER,I,u-pTStsK71w26dj9KuUFbaTyTM2XDKyvvNw75sjxnILzu-t1,PAPA Mo PANOT,22,4,9,False,False,False,False,NaN
4,c3e3c5aa-1277-4b99-97ce-c0365af4962b,RANKED_SOLO_5x5,SILVER,I,cuoCLtlZeX3xx_iMLM0eEVwHLenm6DPAmi0RvIXkpvftm8PL,BeLikeWater04,15,68,50,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,29a44aa3-febc-42c1-94c8-5952a1ade17d,RANKED_SOLO_5x5,GOLD,IV,EF0nFSojEksClGg0Z8ob1M9966fPQSsEwErMurU4NummWDm1,Yuzu Nanami,2,88,107,False,False,False,False,NaN
1226,2c5f54ec-eea9-43fd-8bdf-e051c44a4f88,RANKED_SOLO_5x5,GOLD,IV,Ia7ZZtm1n32o0S4uI_iiu7glEyxsVDT_FgGh3JceO6jw5vOi,k9s,19,204,175,False,False,False,False,NaN
1227,e84892b0-f5c2-47ba-a80a-0d3ed70407c0,RANKED_SOLO_5x5,GOLD,IV,eZ76F1RhkIJIyDdWAikFQfb-amuJcCr69SJrvxr7rzmZxPJg,Cqoox2,85,69,81,False,False,False,True,NaN
1228,eb1d530b-ff9d-44bb-81b9-1328de829fe9,RANKED_SOLO_5x5,GOLD,IV,WK1fJWHLXewWC1Vum7Sr527omkvgiOZsXrQMTFD8gMJo_H5Y,I WANT JOB,58,410,413,False,False,False,False,NaN


In [ ]:
#Selecting two subsets of data - for this section, I decided to use League Points to create the first subset of players. Because this variable is measured from 0 points to 100, 
#I decided to split it up from users with league points 0-50. The second subset, I decided, would be players with over 100 wins. This variable is chosen because it is also numerical,
#and I thought it would be interesting to see if there are any players that have that many wins, yet are still in the lower half of their division.
#In other words, S1 is league points less than 50. S2 is wins greater than 100

new_points = total[total['leaguePoints']<=50]
df1 = new_points.drop(['leagueId',	'queueType',	'tier', 	'rank',	'summonerId',	'wins',	'losses',	'veteran',	'inactive',	'freshBlood',	'hotStreak',	'miniSeries'], axis=1)
new_wins = total[total['wins']>100]
df2 = new_wins.drop(['leagueId',	'queueType',	'tier', 	'rank',	'summonerId',	'leaguePoints',	'losses',	'veteran',	'inactive',	'freshBlood',	'hotStreak',	'miniSeries'], axis=1)
df1


,summonerName,leaguePoints
1,Lowfyz,44
2,Subt3rfug3,23
3,PAPA Mo PANOT,22
4,BeLikeWater04,15
6,TTV ErosXTFT,0
...,...,...
1222,Crazy StoneD 420,0
1223,Uwu Lady,47
1225,Yuzu Nanami,2
1226,k9s,19


In [ ]:
#This section will be where I perform a merge to find the intersection between df1 and df2, which I will do using pd.merge, since the default join is an inner join,
#where both columns must be valid

df_intersection = pd.merge(df1, df2, on='summonerName')
df_intersection

,summonerName,leaguePoints,wins
0,lMarvisl,15,233
1,BE STRONGER W,41,426
2,LickMyChicken,45,174
3,Simops33,5,300
4,bennguyen81,36,830
...,...,...,...
258,Soy Un Bacalao,0,320
259,M3RCKED,0,141
260,Crazy StoneD 420,0,328
261,k9s,19,204


In [ ]:
#This section will be where I perform a merge to find the union between df1 and df2, which I will do using pd.merge, with an outer join, to see the values where either column is valid

df_union = pd.merge(df1, df2, on='summonerName', how='outer', keys=['original', 'new'])
df_union


,summonerName,leaguePoints,wins
0,Lowfyz,44.0,NaN
1,Subt3rfug3,23.0,NaN
2,PAPA Mo PANOT,22.0,NaN
3,BeLikeWater04,15.0,NaN
4,TTV ErosXTFT,0.0,NaN
...,...,...,...
960,Boomstick90001,NaN,109.0
961,639 5 15 11 377,NaN,161.0
962,ILIKETURTLES696,NaN,109.0
963,SeanpoooKEKE,NaN,143.0
